In [1]:
using MixedLayerModel
using FileIO
using Plots
using Revise

include("mlm_solve_funcs.jl")

┌ Info: Precompiling MixedLayerModel [45e51734-b38e-40e9-bdad-5745a9e5fe6d]
└ @ Base loading.jl:1342


run_mlm_ss_from_init

In [8]:
# use command line argument to set co2
newCO2 = 1600.0

# load initial condition from file
path = "output/test_cloudfrac/";
restarttry = path*"co2_upstep_"*string(Int(newCO2-200))*".jld2";
restarttry2 = path*"co2_upstep_"*string(Int(newCO2-400))*".jld2";
if isfile(restarttry)
    output = load(restarttry);
    println("found file with CO2 = "*string(Int(newCO2-200)));
elseif isfile(restarttry2)
    output = load(restarttry2);
    println("found file with CO2 = "*string(Int(newCO2-400)));
else
    output = load(path*"co2_400.jld2");
    println("no file with CO2 = "*string(Int(newCO2-200)));
end
u0 = output["uf"];
OHU = output["OHU"];
println("restarting from CO2 = "*string(output["p"].CO2));

found file with CO2 = 1400
restarting from CO2 = 1400.0


In [ ]:
# set OHU, increase CO2, let SST evolve and check cloud changes
par = upCO2();
par.Hw = 0.1;
par.OHU = OHU;
par.CO2 = newCO2;
par.etype = enBal();
par.fttype = co2dep();
par.rtype = varRad();
par.stype = varSST();
dt, tmax = 2.0, 5.1;

ENV["GKSwstype"]="nul"
u0, sol = run_mlm_from_init(u0, par, dt=3600.0*dt, tspan=(0.0,3600.0*24.0*tmax));

In [ ]:
t = sol.t / 3600.0 / 24.0;
zi = getindex.(sol.u,1);
hM = getindex.(sol.u,2) * 1e-3;
qtM = getindex.(sol.u,3) * 1e3;
sst = getindex.(sol.u,4);
cf = getindex.(sol.u,5);
S = zeros(length(t));
LHF = zeros(length(t));
zb = zeros(length(t));
ΔR = zeros(length(t));
LWP = zeros(length(t));
for (i,si) in enumerate(S)
    zb[i] = calc_LCL(sol.u[i]);
    S[i] = calc_S(sol.u[i], par);
    LHF[i] = calc_LHF(sol.u[i], par);
    ΔR[i] = calc_cloudtop_RAD(sol.u[i], par, par.rtype);
    LWP[i] = incloud_LWP(sol.u[i]);
end 
plot(size=(1200,800), layout=(5,2), dpi=200, left_margin = 5Plots.mm);
plot!(t, zi, legend=false, subplot=1, ylabel="zi, zb [m]");
plot!(t, zb, legend=false, subplot=1);
plot!(t, hM, legend=false, subplot=2, ylabel="hM [kJ/kg]"); 
plot!(t, qtM, legend=false, subplot=3, ylabel="qtM [g/kg]");
plot!(t, sst, legend=false, subplot=4, ylabel="SST [K]");
plot!(t, cf*1e2, legend=false, subplot=5, ylabel="CF [%]");
plot!(t, LHF, legend=false, subplot=6, ylabel="LHF [W/m2]");
plot!(t, ΔR, legend=false, subplot=7, ylabel="ΔR [W/m2]");
plot!(t, LWP * 1e3, legend=false, subplot=8, ylabel="LWP [g/m2]");
plot!(t, S, legend=false, subplot=9, ylabel="S [-]", xlabel="time [days]");
mkpath(replace(path, "output"=>"figures"));
savefig(replace(path, "output"=>"figures")*"sol"*string(Int(newCO2))*"_t.png");

In [5]:
println(ΔR)

[51.351286013113736, 51.41897357038239, 51.5528048851122, 51.66486548279414, 51.77010188919976, 51.87229233455249, 52.00359416000787, 52.13717621252825, 52.23276489306899, 52.360109794560906, 52.489318839876816, 52.61640807184828, 52.74070736894146, 52.91026539330371, 53.02996781657282, 53.14192222261132, 53.30467150549833, 53.412248139817535, 53.56969856062987, 53.72544348495317, 53.81815533153223, 53.96366081887306, 54.10813833290943, 54.24721815273553, 54.380330429712004, 54.44767129579036, 54.57222340466351, 54.6971093189638, 54.81784092247271, 54.93377551750595, 55.104633643060616, 55.20873485788236, 55.30312945221422, 55.39284438758962, 55.47901318797533, 55.56200076210901, 55.64201310151911, 55.719216712444634, 55.84954533872172, 55.9191081183415, 55.9808653887021, 56.039667259718144, 56.09651660891198, 56.204038889323726, 56.25532047466111, 56.29974957778993, 56.39126650121055, 56.4302531099518, 56.46297543003334, 56.49410360586795, 56.570771272211005, 56.59869059058715, 56.620

In [6]:
Teff = 263.5 + 10.8*log(par.CO2/400.0);

zi = getindex.(sol.u,1);
hM = getindex.(sol.u,2);
qtM = getindex.(sol.u,3);

Tct = temp.(zi,hM,qtM);

function cloud_emissivity(LWP, CF)
    a0 = 0.15; # m^2/g
    ϵc = 1 .- exp.(-a0 * LWP); 
    ϵc = ϵc .* CF;
    return ϵc
end

ϵc_up = cloud_emissivity(LWP*1e3, cf);
println(ϵc_up);

σ_SB = 5.67e-8;
x1 = ϵc_up .* σ_SB .* Tct.^4;
x2 = σ_SB .* Teff.^4;
println(x2)
println(x1)


ΔR2 = x1 .- x2;
println(ΔR2);

[0.9994313072111906, 0.9981938419016535, 0.9981671810675993, 0.9980824149756622, 0.9979830491209934, 0.9978674230535853, 0.9978137278196663, 0.9977552885861183, 0.9976013167489316, 0.9975267829994694, 0.9974464163715052, 0.9973597693626332, 0.9972664910131945, 0.9972879574219327, 0.9971876048784521, 0.9970797031410092, 0.9971005850780682, 0.9969849062147083, 0.9970052822301106, 0.9970252892654214, 0.9969009615711838, 0.9969202566543782, 0.9969392454210609, 0.9969577148509244, 0.9969756495417186, 0.9968417631912199, 0.9968591163445781, 0.9968762812212683, 0.9968930255345485, 0.9969093249419299, 0.9970753693627459, 0.9970904630365194, 0.9971049139857284, 0.9971189822445811, 0.9971327296827912, 0.99714617881549, 0.997159343728095, 0.9971722361554258, 0.9973251423236703, 0.9973372611625626, 0.9973488965658941, 0.9973602784232769, 0.9973714549524182, 0.9975140261136392, 0.9975246227021559, 0.9975348120109383, 0.9976684110092747, 0.9976780862714724, 0.9976874020801223, 0.9976965526222147, 0.